<a href="https://colab.research.google.com/github/EvanWAppel/work-examples/blob/main/machineList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries
import os
import pandas as pd
%load_ext google.colab.data_table
from google.colab import data_table
import numpy as np
from google.colab import files
import datetime as datetime
import plotly.express as px
from google.colab import drive
drive.mount('/gdrive')
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default 
creds,_ = default()
gc = gspread.authorize(creds)

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)


Mounted at /gdrive


In [ ]:
#This is for BA
worksheet = gc.open('BitAccess').sheet1
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# Convert to a DataFrame and render.
data = pd.DataFrame.from_records(rows)

#this takes a row of dataframe and converts to a list 
mylist = data.loc[0, :].values.tolist()
#This tells the dataframe that the list of headers we're passing it are the headers
data.columns = mylist
#gets rid of first row
data = data.drop(0)

#This is for Vision
worksheet2 = gc.open('Vision Cashbox Totals').sheet1
# get_all_values gives a list of rows.
rows2 = worksheet2.get_all_values()
# Convert to a DataFrame and render.
vision = pd.DataFrame.from_records(rows2)

mylist = vision.loc[0, :].values.tolist()
vision.columns = mylist
vision = vision.drop(0)

# Master Install Sheet
worksheet3 = gc.open('Master Installs').worksheet('Master Installs')
rows3 = worksheet3.get_all_values()
installs = pd.DataFrame.from_records(rows3)

mylist = installs.loc[0, :].values.tolist()
installs.columns = mylist
installs = installs.drop(0)

# Armored WIP 2021
worksheet4 = gc.open('Armored WIP 2021').worksheet('WIP')
rows4 = worksheet4.get_all_values()
wip = pd.DataFrame.from_records(rows4)

mylist = wip.loc[1, :].values.tolist()
wip.columns = mylist
wip = wip.drop(0)
wip = wip.drop(1)




In [ ]:
# Vision transaction Data
# proto-function for retrieving and concatenating all files from a folder in Google Drive
# Declare directory to retrieve files from
dir = "/gdrive/MyDrive/Cash_Logistics_Process/Transaction_Data/Vision/"
# Get a list of the files and call it "entries"
entries = [entry for entry in os.listdir(dir) if os.path.isfile(os.path.join(dir, entry))]
# Declare an empty dataframe with a target name to put resulting data
visionData = pd.DataFrame()
# loop through entries...
for entry in entries: 
  try:
    # And if they're CSVs...
    if entry.endswith(".csv"):
      # Put them in the placeholder variable...
      placeholder = pd.read_csv("/gdrive/MyDrive/Cash_Logistics_Process/Transaction_Data/Vision/"+entry, parse_dates=True)
    else:
      # use a slightly different function to put them in the placeholder function...
      placeholder = pd.read_xlsx("/gdrive/MyDrive/Cash_Logistics_Process/Transaction_Data/Vision/"+entry, parse_dates=True)
      # Then, collect them in your variable
    visionData = visionData.append(placeholder)
  except:
    print("FILE "+entry+" CANNOT BE READ, PLEASE REPLACE.")
#print(visionData.head())
#######################
# Order Transaction Data

#directory for BA files
dirBA = "/gdrive/MyDrive/Cash_Logistics_Process/Transaction_Data/BA/"
#get a list of the files
entriesBA = [entry for entry in os.listdir(dirBA) if os.path.isfile(os.path.join(dirBA,entry))]

BAData = pd.DataFrame()
#loop through entries
for entry in entriesBA:
  try:
    if entry.endswith(".csv"):
      placeholder = pd.read_csv("/gdrive/MyDrive/Cash_Logistics_Process/Transaction_Data/BA/"+entry, parse_dates=True)
    else:
      placeholder = pd.read_xlsx("/gdrive/MyDrive/Cash_Logistics_Process/Transaction_Data/BA/"+entry,parse_dates=True)
    BAData = BAData.append(placeholder)
  except:
     print("File"+entry+"CANNOT BE READ, PLEASE REPLACE.") 

print(BAData.head())
#


                 ID Short Code                                       Key       Customer ID BTM Machine Name    Kind      State  Amount Deposited Deposit Currency  Requested Withdrawal Amount  Actual Withdrawal Amount Withdrawal Currency                  Withdrawal Address     Price  Spot Price Margin Percentage                                      Blockchain ID  CloudWallet Network Fee Key Org Key Product KYC Level  KYC Limit Deposit  KYC Limit Withdraw  KYC Limit Txns  Bitaccess Fee  Trade ID  Trade Exchange Name  Trade Type  Trade Status  Trade Price  Trade Amount Out  Trade Amount In  Trade Details Fee  Duration          Created At Reference Order BTM Reference Transaction ID  Mining Fee Location ID Promo Code  Promo Discount Altcoin selected Altcoin address        Completed At  \
0  1647820768388093   QYGDOI34  CCLD4947678934859776BTMB1647820768388093  4947678934859776    coincloud1182  btmbuy  completed              70.0              USD                     0.001438               

In [ ]:
# Clean data
# Start with the recylcer in BA
# Select information you want
machines = data[['Machine','Recycler','Cashbox','Location ID','Status']]
# Clean up a column by selecting only digit characters
machines['Recycler'] = machines['Recycler'].str.extract('(\d+)')
# ... Dropping nulls ...
machines = machines.dropna()
# Casting to the a data type
machines['Recycler'] = machines['Recycler'].astype('int64')

# Clean up the Cashbox Column
# Same thing as we just did to the recycler
machines['Cashbox'] = machines['Cashbox'].str.extract('(\d+)')
machines = machines.dropna()
machines['Cashbox'] = machines['Cashbox'].astype('int64')

#Clean up Vision Data
# Select relevant data
vmachines = vision[['Kiosk ID', 'Cash in Recycler', 'Cash in Cash box', 'Location ID']]
# Convert data to appropriate data type.
vmachines['Cash in Recycler'] = vmachines['Cash in Recycler'].astype('int64')
vmachines['Cash in Cash box'] = vmachines['Cash in Cash box'].astype('int64')

# Master Install Sheet
inst = installs[['LID','Installed Date', 'Status']]
inst = inst.rename(columns={'LID':'Location ID',
                            'Status':'Install Status'})
inst = inst.groupby(['Location ID','Install Status'],as_index=False).agg({'Installed Date':max})

# Armored WIP
w = wip[['LID','Loomis','Brinks','Awarded To','Awarded Date','Armored Service']]
w = w.rename(columns={'LID':'Location ID'})
w = w.groupby(['Location ID','Loomis','Brinks','Awarded To','Armored Service'],as_index=False).agg({'Awarded Date':max})

# Vision
print(visionData.head())
# Select relevant columns
v = visionData[['Location Id','Kind','Status','Amount Deposited USD','Cash Timestamp']]
# Convert datetime to datetime
v['Cash Timestamp'] = pd.to_datetime(v['Cash Timestamp'])
v['Cash Timestamp'] = v['Cash Timestamp'].dt.date
# Filter columns so it's only completed buys within the past 30 days
from datetime import timedelta
from datetime import date
v = v[(v['Kind']=='BUY')&(v['Status']=='COMPLETE')&(v['Cash Timestamp']>= date.today() - timedelta(days = 1))]

# rename columns in preparation of join
v = v.rename(columns={'Location Id':'Location ID',
                      'Amount Deposited USD': 'revenue'})
v = v[['Location ID','revenue']]
#BA
#print(BAData.head())

BA = BAData[['Location ID','Kind','State','Amount Deposited','Created At']]

# Convert datetime to datetime - changed object to datetime
BA['Created At'] = pd.to_datetime(BA['Created At'])

#this is to only get date 
BA['Created At'] = BA['Created At'].dt.date

# Filter columns so it's only completed buys within the past 30 days
from datetime import timedelta
from datetime import date
BA = BA[(BA['Kind']=='btmbuy')&(BA['State']=='completed')&(BA['Created At']>= date.today() - timedelta(days = 30))]

BA = BA.groupby('Location ID',as_index=False).agg({'Amount Deposited':sum})
BA = BA.rename(columns={'Amount Deposited':'revenue'})
BA = BA[['Location ID','revenue']]

table = pd.concat([v,BA])

table = table.groupby(['Location ID'],as_index=False).agg({'revenue':sum})

print(table.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

   Transaction Id  Customer Id   Machine Id  Kind    Status Location Id State  Amount Deposited USD  Actual Amount BTC  Amount Deposited BTC  Requested Amount USD  Actual Amount USD  Amount to be Refunded  Alt Coins Amount Alt Coins Currency  Spot Price  Transaction Price  Alt Coin Spot Price  Alt Coin Transaction Price Promo Code  Margin Percentage                              Wallet Address                                      Blockchain ID  Network Fees  Mining Fees BTC           Cash Timestamp           Sell Timestamp       Complete Timestamp  Gross Revenue  Transaction Gross Profit  Mining Fee USD  Machine Gross Profit  Refund Amount USD  Refund Amount BTC  KYC Tier
0  1643727273c1cd  17022393919  Vision03_HQ  SELL  COMPLETE      000000    NV                   NaN                NaN              0.001558                  60.0               60.0                    NaN               NaN                NaN   38644.750       38644.750000                  NaN                         Na

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

  Location ID  Amount Deposited
0     98930.0             150.0
1    101396.0           10170.0
2    101407.0              80.0
3    101427.0              28.0
4    101428.0              20.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# Union tables
vmachines = vmachines.rename(columns={'Kiosk ID':'Machine','Cash in Recycler':'Recycler', 'Cash in Cash box' : 'Cashbox'})
allmachines = pd.concat([machines,vmachines])
allmachines = allmachines[allmachines['Status']!='Inactive']
allmachines = allmachines[allmachines['Location ID']!='']

# Joins


allmachines = pd.merge(allmachines,inst,how = 'left',on = 'Location ID')


allmachines = pd.merge(allmachines,w,how='left',on = 'Location ID')
print(allmachines.head())
print(allmachines.info())


        Machine  Recycler  Cashbox Location ID   Status Installed Date Install Status  Loomis  Brinks Awarded To Armored Service Awarded Date
0   coincloud62         0      142      103695   Online            NaN            NaN     NaN     NaN        NaN             NaN          NaN
1   coincloud66      1200      481      104161   Online            NaN            NaN     NaN     NaN        NaN             NaN          NaN
2   coincloud90      1000     2740      104221   Online            NaN            NaN     NaN     NaN        NaN             NaN          NaN
3  coincloud306         0        0      130607  Offline            NaN            NaN     NaN     NaN        NaN             NaN          NaN
4  coincloud320        80        0      101402   Online            NaN            NaN  $30.00  $32.51                                        
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5717 entries, 0 to 5716
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dty

Delivery

In [ ]:
# Save and Print

from datetime import date
today = date.today()
print(today)

allmachines.to_csv('/gdrive/MyDrive/Cash_Logistics_Process/Daily Cashbox Report/Daily Reports/allmachines' + str(today) +'.csv') 
files.download('/gdrive/MyDrive/Cash_Logistics_Process/Daily Cashbox Report/Daily Reports/allmachines' + str(today) +'.csv')

# save to the daily reports file with the date appended to the name.



2022-03-22


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>